<a href="https://colab.research.google.com/github/ElBenjaM/Whisper-V3/blob/main/Whisper_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

****IMPORTANTE*** tener activada la **Aceleración por hardware** con GPU en `"≡"(en smartphone) > "Entorno de ejecución" > "Cambiar tipo de entorno de ejecución"`.



In [ ]:
#@title ⚙️ Descargar librerias para el funcionamiento del codigo (Obligatorio)
#@markdown Todo es 100% online, ya que el codigo funciona en un "PC" prestado por Google, lo único que se les puede descargar en su PC o Teléfono es la transcripción e interpretación de los Audios/Videos si apretan la sección "Descargar transcripción en archivo Word" o "Descargar Interpretación de GTP-3"
print("Descargando...")
!pip install ffmpeg
!pip install -q git+https://github.com/openai/whisper.git
!pip install openai > /dev/null
!pip install pytube > /dev/null
!pip install python-docx > /dev/null
import docx
import whisper
from google.colab import files
from IPython.display import clear_output
model = whisper.load_model("large-v3")

import openai
import os

if not os.path.exists("audios"):
  os.makedirs("audios")

if not os.path.exists("videos"):
  os.makedirs("videos")

#@title Configuaracion para grabar audio
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
  var my_div = document.createElement("DIV");
  var my_p = document.createElement("P");
  var my_btn = document.createElement("BUTTON");
  var t = document.createTextNode("Press to start recording");

  my_btn.appendChild(t);
  //my_p.appendChild(my_btn);
  my_div.appendChild(my_btn);
  document.body.appendChild(my_div);

  var base64data = 0;
  var reader;
  var recorder, gumStream;
  var recordButton = my_btn;

  var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
      //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
      mimeType : 'audio/webm;codecs=opus'
      //mimeType : 'audio/webm;codecs=pcm'
    };
    //recorder = new MediaRecorder(stream, options);
    recorder = new MediaRecorder(stream);
    recorder.ondataavailable = function(e) {
      var url = URL.createObjectURL(e.data);
      var preview = document.createElement('audio');
      preview.controls = true;
      preview.src = url;
      document.body.appendChild(preview);

      reader = new FileReader();
      reader.readAsDataURL(e.data);
      reader.onloadend = function() {
        base64data = reader.result;
        //console.log("Inside FileReader:" + base64data);
      }
    };
    recorder.start();
  };

  recordButton.innerText = "Grabando... presione para detener";

  navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


  function toggleRecording() {
    if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      //recordButton.innerText = "Guardando Audio..."
      recordButton.parentNode.removeChild(recordButton);
    }
  }

  // https://stackoverflow.com/a/951057
  function sleep(ms) {
    return new Promise(resolve => setTimeout(resolve, ms));
  }

  var data = new Promise(resolve=>{
  //recordButton.addEventListener("click", toggleRecording);
  recordButton.onclick = ()=>{
  toggleRecording()

  sleep(2000).then(() => {
    // wait 2000ms for the data to be available...
    // ideally this should use something like await...
    //console.log("Inside data:" + base64data)
    resolve(base64data.toString())

  });

  }
  });

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr
clear_output()
print("Descargado!")
print("hola mundo")

Despliega el módulo que te interese usar y ve apretando los ▶️ de arriba hacia abajo".

#1-Codigo para subir Audios/Videos y transcribirlos

In [ ]:
#@title Dale al play para comenzar a subir tu Audio o Video
from google.colab import files
import os

# Carga el archivo de audio
audioo = files.upload()

# Accede al primer elemento del diccionario (el nombre del archivo)
Nombre_audio = list(audioo.keys())[0]

In [ ]:
#@title Elegir Modelo
#@markdown Entre más pesado el modelo, saldrá de mejor calidad la transcripción pero se demorará más tiempo, se recomienda usar el modelo "medium" para la mayoria de casos. Si al usar "medium" faltan palabras o transcribe mal usar "large"(el resto mala calidad).

# Mostramos un mensaje de que se está realizando la transcripción
print("Transcribiendo...")

import whisper
modelo_trascripcion = "large-v3" #@param ["tiny", "base", "small", "medium", "large","large-v3"]
model = whisper.load_model(modelo_trascripcion)
audio = whisper.load_audio(f"{Nombre_audio}")

# Convertimos el audio a espectrograma mel-log
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# Realizamos la transcripción
result = model.transcribe(audio, without_timestamps=True)

# Mostramos el texto transcrito por pantalla
print(result["text"])

#------------------------------------------------------------

In [ ]:
#@title Descargar transcripción en archivo Word
import docx
from google.colab import files
# Creamos un nuevo documento de Word
document = docx.Document()

# Añadimos el contenido de la transcripción al documento de Word
document.add_paragraph(f"▼ Transcripción de {Nombre_audio}")
document.add_paragraph(result["text"])
nombr3 = "Transcripcion_"

# Guardamos el documento de Word en la carpeta base de Google Colab
document.save(f"{nombr3+Nombre_audio}.docx")

# Descargamos el documento de Word
files.download(f"{nombr3+Nombre_audio}.docx")

#2-Codigo para grabar audios y transcribirlos

In [ ]:
#@title Grabar Audio
# Importa la librería y los módulos de ipywidgets
import numpy as np
from scipy.io.wavfile import write
from IPython.display import clear_output
from ipywidgets import Button, HBox, VBox, Text

# Crea una caja de texto para ingresar el nombre del audio
audio_input = Text(placeholder='¿Cómo quieres que se llame tu audio?')

# Crea un botón para descargar el audio
download_button = Button(description='Grabar Audio')


# Define la función que se ejecutará al hacer clic en el botón
def on_button_clicked(b):
    clear_output()
    # Obtiene el nombre del audio de la caja de texto
    audio_nombre = audio_input.value +".wav"

    # Grabamos el audio
    audio, sr = get_audio()

    # Almacenamos el audio en un archivo .wav
    write("audios/" + audio_nombre, sr, audio)
    print("  "+audio_nombre + " se ha guardado")

# Asigna la función al evento "on_click" del botón
download_button.on_click(on_button_clicked)

# Muestra la caja de texto y el botón en el notebook
display(HBox([audio_input, download_button]))

In [ ]:
#@title Elegir Modelo
#@markdown Entre más pesado el modelo, saldrá de mejor calidad la transcripción pero se demorará más tiempo, se recomienda usar el modelo "medium" para la mayoria de casos. Si al usar "medium" faltan palabras o transcribe mal usar "large"(el resto mala calidad).

import os

# Obtiene la lista de archivos en la carpeta "audios"
files = os.listdir("audios")

# Ordena los archivos por fecha de modificación
files.sort(key=lambda x: os.path.getmtime("audios/" + x))

# Obtiene el nombre del último archivo
nombre_audio2 = files[-1]

# Mostramos un mensaje de que se está realizando la transcripción
print("Transcribiendo...")

import whisper
modelo_trascripcion = "large-v3" #@param ["tiny", "base", "small", "medium", "large","large-v3"]
model = whisper.load_model(modelo_trascripcion)
audio = whisper.load_audio(f"audios/{nombre_audio2}")

# Convertimos el audio a espectrograma mel-log
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# Realizamos la transcripción
result2 = model.transcribe(audio, without_timestamps=True)

# Mostramos el texto transcrito por pantalla
print(result2["text"])

#------------------------------------------------------------

In [ ]:
#@title Descargar transcripción en archivo Word
import docx
from google.colab import files
# Creamos un nuevo documento de Word
document = docx.Document()

# Añadimos el contenido de la transcripción al documento de Word
document.add_paragraph(f"▼ Transcripción de {nombre_audio2}")
document.add_paragraph(result2["text"])

nombr3 = "Transcripcion_"

# Guardamos el documento de Word en la carpeta base de Google Colab
document.save(f"{nombr3+nombre_audio2}.docx")

# Descargamos el documento de Word
files.download(f"{nombr3+nombre_audio2}.docx")

#3-Codigo transcribir videos de Youtube

In [ ]:
#@title Ingresar URL del video

# Importa la librería y los módulos de ipywidgets
import pytube
from ipywidgets import Button, HBox, VBox, Text
from IPython.display import clear_output

# Crea una caja de texto para ingresar la URL del video
url_input = Text(placeholder='Ingresa la URL del video de YouTube')

# Crea un botón para descargar el video
download_button = Button(description='Descargar video')

# Define la función que se ejecutará al hacer clic en el botón
def on_button_clicked(b):
    # Obtiene la URL del video de la caja de texto
    url = url_input.value

    # Crea un objeto "Video" utilizando la URL del video
    video = pytube.YouTube(url)
    print("Descargando.....(Espera mensaje de confirmación)")
    # Obtiene el primer stream de video disponible en calidad máxima
    stream = video.streams.get_highest_resolution()

    # Descarga el stream a la carpeta "videos"
    stream.download("videos")
    clear_output()
    # Muestra un mensaje de éxito
    print("El video se ha descargado con éxito")

# Asigna la función al evento "on_click" del botón
download_button.on_click(on_button_clicked)

# Muestra la caja de texto y el botón en el notebook
display(HBox([url_input, download_button]))



In [ ]:
#@title Elegir Modelo
#@markdown Entre más pesado el modelo, saldrá de mejor calidad la transcripción pero se demorará más tiempo, se recomienda usar el modelo "medium" para la mayoria de casos. Si al usar "medium" faltan palabras o transcribe mal usar "large"(el resto mala calidad).
# Mostramos un mensaje de que se está realizando la transcripción
import os

# Obtiene la lista de archivos en la carpeta "videos"
files = os.listdir("videos")

# Ordena la lista por fecha de modificación (del más reciente al más antiguo)
files = sorted(files, key=lambda x: os.path.getmtime("videos/"+x))

# Obtiene el último archivo de la lista (el más reciente)
nombre_audio3 = files[-1]

print("Transcribiendo...")

import whisper
modelo_trascripcion = "large-v3" #@param ["tiny", "base", "small", "medium", "large","large-v3"]
model = whisper.load_model(modelo_trascripcion)
audio = whisper.load_audio(f"videos/{nombre_audio3}")

# Convertimos el audio a espectrograma mel-log
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# Realizamos la transcripción
result3 = model.transcribe(audio, without_timestamps=True)

# Mostramos el texto transcrito por pantalla
print(result3["text"])

#------------------------------------------------------------

In [ ]:
#@title Descargar transcripción en archivo Word
import docx
from google.colab import files

# Creamos un nuevo documento de Word
document = docx.Document()

# Añadimos el contenido de la transcripción al documento de Word
document.add_paragraph(f"▼ Transcripción de {nombre_audio3}")
document.add_paragraph(result3["text"])

nombr3 = "Transcripcion_"

# Guardamos el documento de Word en la carpeta base de Google Colab
document.save(f"{nombr3+nombre_audio3}.docx")

# Descargamos el documento de Word
files.download(f"{nombr3+nombre_audio3}.docx")